In [1]:
from neo4j import GraphDatabase

class Neo4j:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

neo4j = Neo4j("bolt://localhost:7687", "neo4j", "password")

ModuleNotFoundError: No module named 'neo4j'

In [ ]:
# neo4j.driver.execute_query("MATCH (p) RETURN p LIMIT 2")

def execute(query):
    with neo4j.driver.session() as session:
        result = session.run(query)
        return result

# Import Data


## Queries


In [17]:
clean_up = "MATCH (n) DETACH DELETE n"
show_all = "MATCH (n) RETURN n"

# Import files: /Users/enioohmaye/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-41d35f05-e4e0-4f5a-a04e-64b41bb80839/import

# Load SPINs
import_students = """
        LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
        WITH row WHERE row.FirstName IS NOT NULL
        MERGE (s:Student {id: apoc.create.uuid(), firstName:row.FirstName, lastName:row.LastName})
"""

import_SPINs = """
        LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
        WITH row WHERE row.{0} IS NOT NULL
        MERGE (s:SPIN {{name:row.{0}}})
        ON CREATE SET s.id = apoc.create.uuid()
"""

import_student_WANTS_spins = """
        LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
        WITH row WHERE row.FirstName IS NOT NULL
        MATCH (stud:Student) WHERE row.FirstName = stud.firstName and row.LastName = stud.lastName
        MATCH (spin:SPIN) WHERE spin.name = row.{0}
        CREATE (stud)-[wants:WANTS {{ranking:{1}, choice:'{0}'}}]->(spin)
"""


import_student_level = """
        LOAD CSV WITH HEADERS FROM 'file:///2_Weekly_Student_Class.csv' AS row
        WITH row WHERE row.`First Name` IS NOT NULL
        MATCH (stud:Student) WHERE row.`First Name` = stud.firstName and row.`Last Name` = stud.lastName
        SET stud.level = row.Level
"""
import_rooms = """
        LOAD CSV WITH HEADERS FROM 'file:///Rooms.csv' AS row
        WITH row WHERE row.Room IS NOT NULL
        MERGE (r:Room {id: apoc.create.uuid(), name:row.Room, capacity:coalesce(row.Capacity, 'Unknown'), type:coalesce(row.Type,'Unknown')})
"""
import_blocks = """
        LOAD CSV WITH HEADERS FROM 'file:///Blocks.csv' AS row
        WITH row WHERE row.Block IS NOT NULL
        MERGE (r:Block {id: apoc.create.uuid(), name:row.Block, startTime: row.startTime, endTime: row.endTime})
"""

import_teachers = """
        LOAD CSV WITH HEADERS FROM 'file:///AllTeachersList2024Jan.csv' AS row
        WITH row WHERE row.nameRomaji IS NOT NULL
        MERGE (r:Teacher {id: randomUUID(), name:row.nameRomaji, 
        nameJP:row.nameJP, SPINStrength:coalesce(row.SPINStrength, ""), email:row.email})
"""

In [18]:
execute(clean_up)
# execute(show_all)
execute(import_students)
execute(import_SPINs.format("Choice01"))
execute(import_SPINs.format("Choice02"))
execute(import_SPINs.format("Choice03"))
execute(import_SPINs.format("Choice04"))
execute(import_SPINs.format("Choice05"))
execute(import_SPINs.format("IntensiveChoice01"))
execute(import_SPINs.format("IntensiveChoice02"))
execute(import_SPINs.format("IntensiveChoice03"))
execute(import_student_WANTS_spins.format("Choice01", 10))
execute(import_student_WANTS_spins.format("Choice02", 8))
execute(import_student_WANTS_spins.format("Choice03", 6))
execute(import_student_WANTS_spins.format("Choice04", 4))
execute(import_student_WANTS_spins.format("Choice05", 2))
execute(import_student_WANTS_spins.format("IntensiveChoice01", 10))
execute(import_student_WANTS_spins.format("IntensiveChoice02", 8))
execute(import_student_WANTS_spins.format("IntensiveChoice03", 6))
execute (import_student_level)
execute(import_rooms)
execute(import_blocks)
execute(import_teachers)

# Queries


In [ ]:
# Top choices sorted by number of students choosing it as Choice01
q = """
    MATCH (stud:Student)-[r:WANTS]->(spin)
    WITH spin, r.ranking as rank, stud.level as level 
    RETURN count(*) as Total, level, rank, spin ORDER BY rank DESC,Total DESC
"""
records, summary, keys = neo4j.driver.execute_query(q)
for record in records:
    print(record.data())

In [16]:
q = """
    MATCH (stud:Student)-[r:WANTS]->(spin:SPIN)
    WITH spin,  count(r.ranking = 10) as Choice01, count(r.ranking = 8) as Choice02, 
    count(r.ranking = 6) as Choice03, count(r.ranking = 4) as Choice04, count(r.ranking = 2) as Choice05
    RETURN  spin, Choice01, Choice02, Choice03, Choice04, Choice05, 
    (Choice01+ Choice02 + Choice03 + Choice04 + Choice05) as Score ORDER BY Score DESC
"""
records, summary, keys = neo4j.driver.execute_query(q)
for record in records:
    print(record.data())

{'spin': {'SPIN': '*Communication(A1-B2)'}, 'Choice01': 130, 'Choice02': 130, 'Choice03': 130, 'Choice04': 130, 'Choice05': 130, 'Score': 650}
{'spin': {'SPIN': '*Learning Japanese w/ Games and Board Games(A1-B2)'}, 'Choice01': 129, 'Choice02': 129, 'Choice03': 129, 'Choice04': 129, 'Choice05': 129, 'Score': 645}
{'spin': {'SPIN': 'ことば（Vocabulary ）'}, 'Choice01': 123, 'Choice02': 123, 'Choice03': 123, 'Choice04': 123, 'Choice05': 123, 'Score': 615}
{'spin': {'SPIN': 'リスニング&アニメ (Listening and Anime)'}, 'Choice01': 120, 'Choice02': 120, 'Choice03': 120, 'Choice04': 120, 'Choice05': 120, 'Score': 600}
{'spin': {'SPIN': '*JLPT Reading(A1.3-B2)'}, 'Choice01': 119, 'Choice02': 119, 'Choice03': 119, 'Choice04': 119, 'Choice05': 119, 'Score': 595}
{'spin': {'SPIN': '生活の中の読解 Reading in Everyday Life (A1-B1)'}, 'Choice01': 118, 'Choice02': 118, 'Choice03': 118, 'Choice04': 118, 'Choice05': 118, 'Score': 590}
{'spin': {'SPIN': 'Experience'}, 'Choice01': 96, 'Choice02': 96, 'Choice03': 96, 'Choice

In [22]:
q = """
    MATCH (stud:Student)-[r:WANTS]->(spin:SPIN)
    WITH spin, count(CASE WHEN r.ranking = 10 THEN 1 ELSE NULL END) as Choice01, 
    count(CASE WHEN r.ranking = 8 THEN 1 ELSE NULL END) as Choice02, 
    count(CASE WHEN r.ranking = 6 THEN 1 ELSE NULL END) as Choice03, 
    count(CASE WHEN r.ranking = 4 THEN 1 ELSE NULL END) as Choice04, 
    count(CASE WHEN r.ranking = 2 THEN 1 ELSE NULL END) as Choice05
    RETURN spin, Choice01, Choice02, Choice03, Choice04, Choice05, 
    (Choice01 + Choice02 + Choice03 + Choice04 + Choice05) as Score ORDER BY Score DESC
"""
records, summary, keys = neo4j.driver.execute_query(q)
for record in records:
    print(record.data())

{'spin': {'SPIN': '*Communication(A1-B2)'}, 'Choice01': 47, 'Choice02': 37, 'Choice03': 46, 'Choice04': 0, 'Choice05': 0, 'Score': 130}
{'spin': {'SPIN': '*Learning Japanese w/ Games and Board Games(A1-B2)'}, 'Choice01': 59, 'Choice02': 31, 'Choice03': 39, 'Choice04': 0, 'Choice05': 0, 'Score': 129}
{'spin': {'SPIN': 'ことば（Vocabulary ）'}, 'Choice01': 26, 'Choice02': 26, 'Choice03': 25, 'Choice04': 21, 'Choice05': 25, 'Score': 123}
{'spin': {'SPIN': 'リスニング&アニメ (Listening and Anime)'}, 'Choice01': 41, 'Choice02': 21, 'Choice03': 18, 'Choice04': 16, 'Choice05': 24, 'Score': 120}
{'spin': {'SPIN': '*JLPT Reading(A1.3-B2)'}, 'Choice01': 23, 'Choice02': 57, 'Choice03': 39, 'Choice04': 0, 'Choice05': 0, 'Score': 119}
{'spin': {'SPIN': '生活の中の読解 Reading in Everyday Life (A1-B1)'}, 'Choice01': 18, 'Choice02': 36, 'Choice03': 30, 'Choice04': 24, 'Choice05': 10, 'Score': 118}
{'spin': {'SPIN': 'Experience'}, 'Choice01': 10, 'Choice02': 21, 'Choice03': 22, 'Choice04': 17, 'Choice05': 26, 'Score': 96

In [18]:
q = """
    MATCH (stud:Student)-[r:WANTS]->(spin:SPIN)
    RETURN spin,
    sum(CASE WHEN r.ranking = 10 THEN 1 ELSE 0 END) as Count10,
    sum(CASE WHEN r.ranking = 8 THEN 1 ELSE 0 END) as Count8,
    sum(CASE WHEN r.ranking = 6 THEN 1 ELSE 0 END) as Count6,
    sum(CASE WHEN r.ranking = 4 THEN 1 ELSE 0 END) as Count4,
    sum(CASE WHEN r.ranking = 2 THEN 1 ELSE 0 END) as Count2
"""
records, summary, keys = neo4j.driver.execute_query(q)
for record in records:
    print(record.data())

{'spin': {'SPIN': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）'}, 'Count10': 29, 'Count8': 20, 'Count6': 21, 'Count4': 18, 'Count2': 5}
{'spin': {'SPIN': '作文＆クリエイティブライティング（Basic＆Creative Writing）'}, 'Count10': 9, 'Count8': 6, 'Count6': 18, 'Count4': 20, 'Count2': 20}
{'spin': {'SPIN': 'JLPT文法 (JLPT Grammar) (A1.3-B2)'}, 'Count10': 24, 'Count8': 13, 'Count6': 9, 'Count4': 22, 'Count2': 24}
{'spin': {'SPIN': 'リスニング&アニメ (Listening and Anime)'}, 'Count10': 41, 'Count8': 21, 'Count6': 18, 'Count4': 16, 'Count2': 24}
{'spin': {'SPIN': 'Experience'}, 'Count10': 10, 'Count8': 21, 'Count6': 22, 'Count4': 17, 'Count2': 26}
{'spin': {'SPIN': 'ことば（Vocabulary ）'}, 'Count10': 26, 'Count8': 26, 'Count6': 25, 'Count4': 21, 'Count2': 25}
{'spin': {'SPIN': 'A1 Grammar (A1.2-A1.3)'}, 'Count10': 12, 'Count8': 17, 'Count6': 18, 'Count4': 17, 'Count2': 24}
{'spin': {'SPIN': '生活の中の読解 Reading in Everyday Life (A1-B1)'}, 'Count10': 18, 'Count8': 36, 'Count6': 30, 'Count4': 24, 'Count2': 10}
{'spin': {'SPIN': 'N4 L1-10 

In [26]:
execute(import_rooms)

In [2]:
import uuid
id = uuid.uuid4()
print(id)

f43377ba-0ff1-40bf-887d-53b2ba8336df


In [4]:
from tabulate import tabulate
from py2neo import Graph

# Connect to your Neo4j database
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

# Execute the Cypher query
query = """
MATCH (st:Student)-[w:WANTS]->(sp:SPIN)
WITH st, COLLECT({choice: w.choice, spin: sp.name}) as choices 
RETURN st.id as student_ID, st.firstName as firstName, st.lastName as lastName, choices as choices
ORDER BY st.name
"""
result = graph.run(query)

# Transform the result into a table structure
table_data = []
for record in result:
    student_id = record["student_ID"]
    first_name = record["firstName"]
    last_name = record["lastName"]
    choices = record["choices"]

    # Create a dictionary to represent each row in the table
    row = {"Student ID": student_id, "First Name": first_name, "Last Name": last_name}
    for choice in choices:
        choice_number = choice["choice"]
        course_name = choice["spin"]
        row[f"Choice{choice_number}"] = course_name

    table_data.append(row)

# Generate the table using the tabulate library
table = tabulate(table_data, headers="keys", tablefmt="github")

# Print the table
table_data

[{'Student ID': '330e441a-fbe3-4acd-9a89-c28a6ceb625f',
  'First Name': 'Hannah',
  'Last Name': 'Werksman',
  'ChoiceChoice03': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
  'ChoiceChoice01': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
  'ChoiceChoice04': '作文＆クリエイティブライティング（Basic＆Creative Writing）',
  'ChoiceChoice05': 'A1 Grammar (A1.2-A1.3)',
  'ChoiceChoice02': '生活の中の読解 Reading in Everyday Life (A1-B1)'},
 {'Student ID': '8f725a7d-dc97-4864-ad31-5212e66579f5',
  'First Name': 'Jasmin',
  'Last Name': 'Steiner',
  'ChoiceChoice05': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
  'ChoiceChoice04': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
  'ChoiceChoice03': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
  'ChoiceChoice01': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
  'ChoiceChoice02': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
  'ChoiceIntensiveChoice03': '*JLPT Reading(A1.3-B2)',
  'ChoiceIntensiveChoice02': '*Communication(A1-B2)',
  'ChoiceIntensiveChoice01': '*Learning Japanese w/ Games and Board Games(A1-B2)'},
 {'Studen

In [14]:
{'a': 1}.get('b', '-')

'-'